In [1]:
import time
from bs4 import BeautifulSoup
import sys, io
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.proxy import *

In [2]:
no_of_reviews = 10000

non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
driver = webdriver.Chrome("/Users/reejungkim/Downloads/chromedriver")

wait = WebDriverWait( driver, 10 )

In [3]:
#Append your app store urls here
urls = ["https://play.google.com/store/apps/details?id=com.facebook.katana"]

In [4]:
import pandas as pd
df = pd.DataFrame()

In [5]:
%%time

for url in urls:

    driver.get(url)

    page = driver.page_source

    soup_expatistan = BeautifulSoup(page, "html.parser")

    # open all reviews
    url = url+'&showAllReviews=true'
    driver.get(url)
    time.sleep(5) # wait dom ready
    for i in range(1,10):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')#scroll to load other reviews
        time.sleep(1)
    page = driver.page_source

    soup_expatistan = BeautifulSoup(page, "html.parser")
    expand_pages = soup_expatistan.findAll("div", class_="d15Mdf")
    counter = 1
    for expand_page in expand_pages:
        try:
            
            author = str(expand_page.find("span", class_="X43Kjb").text)
            review_date = expand_page.find("span", class_="p2TkOb").text
            
            reviewer_ratings = expand_page.find("div", class_="jUL89d y92BAb").text
            #print("Reviewer Ratings: ", reviewer_ratings)

            review_body = str(expand_page.find("div", class_="UD7Dzf").text)
            #print("Review Body: ", review_body)
            
            star = expand_page.find("div", class_="pf5lIe").find_next()['aria-label']
            
            developer_reply = expand_page.find_parent().find("div", class_="LVQB0b")

            new_row = {'author':author, 'review_date':review_date, 
                       'reviewer_ratings':reviewer_ratings, 'review':review_body, 'star':star,
                       'developer_reply': developer_reply
                }
            df = df.append(new_row, ignore_index=True)

            counter+=1
        except:
            pass
driver.quit()

CPU times: user 1.14 s, sys: 26.2 ms, total: 1.16 s
Wall time: 19.8 s


In [8]:
import datetime as dt
timestamp = dt.datetime.today().strftime('%Y-%m-%d-%H:%m:%S')

In [9]:
file_name = 'GooglePlayStore_Review_' + timestamp + '.csv'

In [10]:
df.to_csv(file_name, index = True)

driver.quit()